In [ ]:
import os
import numpy as np
import pandas as pd
from scroutines.config_plots import *

In [ ]:
ddir = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt186/proc/r1'
!ls $ddir
# n5dir = '/data/qlyu/v1/data/easifish/lt186/r1/stitching/export.n5'
f1 = os.path.join(ddir, 'roi.csv')
f2 = os.path.join(ddir, 'spotcount.csv') #_intn.csv')

In [ ]:
df1 = pd.read_csv(f1, index_col=0)
df2 = pd.read_csv(f2, index_col=0)
df  = df1.join(df2)

In [ ]:
amin, amax = 500, 5000
cond = ((df['area'] < amax) & (df['area'] > amin))
df = df[cond]

In [ ]:
# cell level stats
fig, axs = plt.subplots(1,4,figsize=(4*4,1*4))
ax = axs[0]
sns.histplot(df['x'].values, ax=ax)
ax.set_xlabel('x')
ax = axs[1]
sns.histplot(df['y'].values, ax=ax)
ax.set_xlabel('y')
ax = axs[2]
sns.histplot(df['z'].values, ax=ax)
ax.set_xlabel('z')
ax = axs[3]
sns.histplot(df['area'].values, ax=ax)
ax.set_xlabel('area')
ax.axvline(amin)
ax.axvline(amax)
fig.tight_layout()
plt.show()

In [ ]:
channels = [
    'r1_c0',  'r1_c2',
]
genes = [
    'RL (Cre)',
    'LM (FlpO)',
]

In [ ]:
# cell level stats
for ch in channels: 
    fig, axs = plt.subplots(1,3,figsize=(3*4,1*4))
    ax = axs[0]
    sns.scatterplot(data=df, x='x', y=ch, ax=ax, s=3, edgecolor='none')
    ax.set_xlabel('x')

    ax = axs[1]
    sns.scatterplot(data=df, x='y', y=ch, ax=ax, s=3, edgecolor='none')
    ax.set_xlabel('y')

    ax = axs[2]
    sns.scatterplot(data=df, x='z', y=ch, ax=ax, s=3, edgecolor='none')
    ax.set_xlabel('z')

    fig.tight_layout()
    plt.show()

In [ ]:
df.describe()

In [ ]:
fig, ax = plt.subplots()
sns.histplot(df['r1_c0'].values, cumulative=True, element='step', fill=False, label='LM', ax=ax)
sns.histplot(df['r1_c2'].values, cumulative=True, element='step', fill=False, label='RL', ax=ax)
ax.axvline(5, linestyle='--', color='k')
ax.axvline(10, linestyle='--', color='k')
ax.axvline(30, linestyle='--', color='k')
ax.legend()

In [ ]:
for ch, gene in zip(channels, genes):
    fig, ax = plt.subplots(figsize=(10,10))
    vmax = np.percentile(df[f'{ch}'], 99.9)
    print(vmax)
    g = ax.scatter(df['x'].values, df['z'].values, c=df[f'{ch}'], s=1, edgecolor='none', 
                   vmax = vmax,
               cmap='rocket_r')
    ax.set_title(gene)
    ax.set_aspect('equal')
    fig.colorbar(g, shrink=0.3)
    plt.show()

In [ ]:
def rename(code):
    rename_dict = {
        0: 'NA', 
        1: 'RL', 
        2: 'LM', 
        3: 'both'}
    return rename_dict[code]

df = df.copy()
th_c0 = 10 
th_c2 = 8 
df['bi_r1_c0'] = df['r1_c0'] > th_c0 
df['bi_r1_c2'] = df['r1_c2'] > th_c2 
df['lbl'] = df['bi_r1_c0'].astype(int) + 2*df['bi_r1_c2'].astype(int)
df['lbl_v2'] = df['lbl'].apply(rename)
np.unique(df['lbl'].values)
np.unique(df['lbl_v2'].values)


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(10,5*2))
for ax, ch, gene in zip(axs, channels, genes):
    g = ax.scatter(df['x'].values, df['z'].values, c=df[f'bi_{ch}'], s=3, edgecolor='none', 
               cmap='rocket_r')
    ax.set_title(gene)
    ax.set_aspect('equal')
    fig.colorbar(g, shrink=0.3)
plt.show()

In [ ]:
sns.color_palette('tab10', 10)

In [ ]:
palette = {
    'NA': 'lightgray', 
    'LM': 'C3', 
    'RL': 'C0', 
    'both': 'C6', 
}

In [ ]:
df.value_counts('lbl_v2')

In [ ]:
a = df.value_counts('lbl_v2')
print(a/len(df)*100, "\n")

print(f"both in LM (%): {a.loc['both']/(a.loc['both']+a.loc['LM'])*100:.1f}")
print(f"both in RL (%): {a.loc['both']/(a.loc['both']+a.loc['RL'])*100:.1f}")
print(f"RL/LM (%): {(a.loc['both']+a.loc['RL'])/(a.loc['both']+a.loc['LM'])*100:.1f}")

In [ ]:
fig, ax = plt.subplots(figsize=(3,4))
df[df['lbl_v2']!='NA'].value_counts('lbl_v2').plot.bar(width=0.7, color='black')
plt.show()

In [ ]:
def plot_easifish_proj(data, x, y, hue='lbl_v2', na_rep='NA', invert_yaxis=False, figsize=(8,8)):
    
    fig, ax = plt.subplots(figsize=figsize)
    sns.scatterplot(data=df, 
                    x=x, y=y, color='lightgray', s=2, edgecolor='none')
    sns.scatterplot(data=df[df[hue]!=na_rep], 
                    x=x, y=y, hue=hue, palette=palette, s=5, edgecolor='none')
    if invert_yaxis:
        ax.invert_yaxis()
        
    ax.grid(False)
    ax.set_aspect('equal')
    ax.legend(bbox_to_anchor=(1,1))
    plt.show()

In [ ]:
plot_easifish_proj(df, 'x', 'y')
plot_easifish_proj(df, 'x', 'z', invert_yaxis=True)
plot_easifish_proj(df, 'y', 'z', invert_yaxis=True)

In [ ]:
xmin, ymin, zmin = df[['x', 'y', 'z']].min()
xmax, ymax, zmax = df[['x', 'y', 'z']].max()
xbins = np.arange(0, xmax+50, 50).astype(int)
ybins = np.arange(0, ymax+50, 50).astype(int)
zbins = np.arange(0, zmax+50, 50).astype(int)
xbins.shape, ybins.shape, zbins.shape

In [ ]:
xmin, ymin, zmin
xmax, ymax, zmax

In [ ]:
df['xbin'] = pd.cut(df['x'], bins=xbins, labels=False) #.astype(int)
df['ybin'] = pd.cut(df['y'], bins=ybins, labels=False)
df['zbin'] = pd.cut(df['z'], bins=zbins, labels=False)
df

In [ ]:
dfsize_x = df.groupby(['xbin', 'lbl_v2']).size().unstack().fillna(0) #mean().unstack()
plt.plot(dfsize_x['LM'], label='LM')
plt.plot(dfsize_x['RL'], label='RL')
plt.plot(dfsize_x['both'], label='both')
plt.legend()

In [ ]:
dfsize_y = df.groupby(['ybin', 'lbl_v2']).size().unstack().fillna(0) #mean().unstack()
plt.plot(dfsize_y['LM'], label='LM')
plt.plot(dfsize_y['RL'], label='RL')
plt.plot(dfsize_y['both'], label='both')
plt.legend()

In [ ]:
dfsize_z = df.groupby(['zbin', 'lbl_v2']).size().unstack().fillna(0) #mean().unstack()
plt.plot(dfsize_z['LM'], label='LM')
plt.plot(dfsize_z['RL'], label='RL')
plt.plot(dfsize_z['both'], label='both')
plt.legend()

In [ ]:
dfsize_xz1 = df[df['lbl_v2']=='LM'].groupby(['xbin', 'zbin']).size().unstack().fillna(0).astype(int) #mean().unstack()
dfsize_xz1 = dfsize_xz1.reindex(np.arange(len(xbins))).T.reindex(np.arange(len(zbins))).T
dfsize_xz1 = dfsize_xz1.fillna(0).astype(int)

dfsize_xz2 = df[df['lbl_v2']=='RL'].groupby(['xbin', 'zbin']).size().unstack().fillna(0).astype(int) #mean().unstack()
dfsize_xz2 = dfsize_xz2.reindex(np.arange(len(xbins))).T.reindex(np.arange(len(zbins))).T
dfsize_xz2 = dfsize_xz2.fillna(0).astype(int)

fig, axs = plt.subplots(1,3,figsize=(3*7,1*5),sharex=True,sharey=True)
ax = axs[0]
sns.heatmap(dfsize_xz1.T, ax=ax, cbar_kws=dict(shrink=0.5))
ax.set_aspect('equal')
ax.set_title('LM')

ax = axs[1]
sns.heatmap(dfsize_xz2.T, ax=ax, cbar_kws=dict(shrink=0.5))
ax.set_aspect('equal')
ax.set_title('RL')

ax = axs[2]
sns.heatmap(np.log2((dfsize_xz1.T+1)/(dfsize_xz2.T+1)), 
            cmap='coolwarm', 
            center=0,
            cbar_kws=dict(shrink=0.5)
           )
ax.set_aspect('equal')
ax.set_title('log2(LM/RL)')
plt.show()

In [ ]:
dfsize_yz1 = df[df['lbl_v2']=='LM'].groupby(['ybin', 'zbin']).size().unstack().fillna(0).astype(int) #mean().unstack()
dfsize_yz1 = dfsize_yz1.reindex(np.arange(len(xbins))).T.reindex(np.arange(len(zbins))).T
dfsize_yz1 = dfsize_yz1.fillna(0).astype(int)

dfsize_yz2 = df[df['lbl_v2']=='RL'].groupby(['ybin', 'zbin']).size().unstack().fillna(0).astype(int) #mean().unstack()
dfsize_yz2 = dfsize_yz2.reindex(np.arange(len(xbins))).T.reindex(np.arange(len(zbins))).T
dfsize_yz2 = dfsize_yz2.fillna(0).astype(int)

fig, axs = plt.subplots(1,3,figsize=(3*7,1*5),sharex=True,sharey=True)
ax = axs[0]
sns.heatmap(dfsize_yz1.T, ax=ax, cbar_kws=dict(shrink=0.5))
ax.set_aspect('equal')
ax.set_title('LM')

ax = axs[1]
sns.heatmap(dfsize_yz2.T, ax=ax, cbar_kws=dict(shrink=0.5))
ax.set_aspect('equal')
ax.set_title('RL')

ax = axs[2]
sns.heatmap(np.log2((dfsize_yz1.T+1)/(dfsize_yz2.T+1)), 
            cmap='coolwarm', 
            center=0,
            cbar_kws=dict(shrink=0.5)
           )
ax.set_aspect('equal')
ax.set_title('log2(LM/RL)')
plt.show()

In [ ]:
dfsize_xy1 = df[df['lbl_v2']=='LM'].groupby(['xbin', 'ybin']).size().unstack().fillna(0).astype(int) #mean().unstack()
dfsize_xy1 = dfsize_xy1.reindex(np.arange(len(xbins))).T.reindex(np.arange(len(zbins))).T
dfsize_xy1 = dfsize_xy1.fillna(0).astype(int)

dfsize_xy2 = df[df['lbl_v2']=='RL'].groupby(['xbin', 'ybin']).size().unstack().fillna(0).astype(int) #mean().unstack()
dfsize_xy2 = dfsize_xy2.reindex(np.arange(len(xbins))).T.reindex(np.arange(len(zbins))).T
dfsize_xy2 = dfsize_xy2.fillna(0).astype(int)

fig, axs = plt.subplots(1,3,figsize=(3*7,1*5),sharex=True,sharey=True)
ax = axs[0]
sns.heatmap(dfsize_xy1.T, ax=ax, cbar_kws=dict(shrink=0.5))
ax.set_aspect('equal')
ax.set_title('LM')

ax = axs[1]
sns.heatmap(dfsize_xy2.T, ax=ax, cbar_kws=dict(shrink=0.5))
ax.set_aspect('equal')
ax.set_title('RL')

ax = axs[2]
sns.heatmap(np.log2((dfsize_xy1.T+1)/(dfsize_xy2.T+1)), 
            cmap='coolwarm', 
            center=0,
            cbar_kws=dict(shrink=0.5)
           )
ax.set_aspect('equal')
ax.set_title('log2(LM/RL)')
plt.show()